In [26]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")


D=3;
chi=5;
W=15;
N=8;
tol=1e-6;
EH_n=3;#number of entanglement spectrum per k point

Dtrun_init=chi*chi*W*W;
Dtrun_max=Dtrun_init;
trun_tol=1e-8;
group_size=Int(round((10^8)/(chi*chi*W*W*D)));


pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [27]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


In [28]:
O1=[];
O2=[];
Ag=[];
try
    #load data
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    Ag_dict=load(jld_filenm)["Ag_dict"];
    O1_dict=load(jld_filenm)["O1_dict"];
    O2_dict=load(jld_filenm)["O2_dict"];
    Ag=convert(TensorMap,Ag_dict);
    O1=convert(TensorMap,O1_dict);
    O2=convert(TensorMap,O2_dict);
    display("load itebd ground state from saved data directly")
catch e

    display("No itebd ground state found from saved data, now do CTMRG and itebd")


    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    @time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);
    
    @time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
    @time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
    display((E_up+E_down)/3)

    
    Tleft=CTM["Tset"][4];
    Tright=CTM["Tset"][2];
    @tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
    @tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];
    
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
    A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

    #Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
    Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

    #save itebd data
    O1_dict=convert(Dict,O1);
    O2_dict=convert(Dict,O2);
    Ag_dict=convert(Dict,Ag);
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

    #save initial CTM to compare with other codes
    matwrite("itebd_matfile.mat", Dict(
        "O1" => convert(Array,O1),
        "O2" => convert(Array,O2),
        "A_init" => convert(Array,A_init),
        "Ag" => convert(Array,Ag)
    ); compress = false)


end

"No itebd ground state found from saved data, now do CTMRG and itebd"

"initialize CTM"

"start CTM iterations:"

-0.3930143570777442 - 1.9464063477406087e-17im

 16.296989 seconds (13.03 M allocations: 711.883 MiB, 2.98% gc time, 98.88% compilation time)
CTMRG iteration: 1, CTMRG err: 1.7185997337254184
CTMRG iteration: 2, CTMRG err: 0.15134704733575097
CTMRG iteration: 3, CTMRG err: 0.24715505015085912
CTMRG iteration: 4, CTMRG err: 0.10810483168320249
CTMRG iteration: 5, CTMRG err: 0.06597229624943744
CTMRG iteration: 6, CTMRG err: 0.27902787235372345
CTMRG iteration: 7, CTMRG err: 0.557267378150516
CTMRG iteration: 8, CTMRG err: 0.02652143908939217
CTMRG iteration: 9, CTMRG err: 0.0024279753380375287
CTMRG iteration: 10, CTMRG err: 0.0007007236010502702
CTMRG iteration: 11, CTMRG err: 0.0002740771483196228
CTMRG iteration: 12, CTMRG err: 0.00010406498427628881
CTMRG iteration: 13, CTMRG err: 3.742155409463203e-5
CTMRG iteration: 14, CTMRG err: 1.2954612067014361e-5
CTMRG iteration: 15, CTMRG err: 4.375372383371749e-6
CTMRG iteration: 16, CTMRG err: 1.4509265927071476e-6
CTMRG iteration: 17, CTMRG err: 4.7431683616187375e-7
175.761932 second

In [29]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));

AOA_sec=collect(sectors(space_AOA))
AA_sec=collect(sectors(space_AA))

@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))

(Rep[SU₂](0=>5, 1/2=>4, 1=>4) ⊗ Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>5, 1/2=>4, 1=>4)' ⊗ Rep[SU₂](0=>1, 1/2=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [30]:
# for s in sectors(space_AOA)
#   display(s)
#   @show s, dim(space_AOA, s), dim(s)
# end


In [31]:
check_complete_decomp=false;
if check_complete_decomp
    Dtrun=W*W*chi*chi;

    euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
    display(size(euL_set))
    eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
    display(size(eul_set))
    S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
    display(size(S_set))
    s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
    display(size(s_set))


    euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
    display(euR_set)
    eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
    display(eur_set)
    S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
    display(S_set)
    s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
    display(s_set)




    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for cc=1:pow-1
                euR=euR*euR0;
            end

            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))




    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for cc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))





    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))



    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

end


In [32]:
euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
# display(euR_set)

eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
# display(eur_set)


"FLR_eig, DTrun=5625, SU2 reduced to 1591, minimal eigenvalue: 9.200844511225464e-12"

"GLR_eig, Dtrun=5625, SU2 reduced to 71, minimal eigenvalue: 4.7346720083208083e-7"

"Dtrun exceeds matrix size"

(Any[ComplexF64[0.9999999999999987 - 3.3306690738754696e-16im, 0.10160177256060615 + 0.7324274047478666im, 0.10160177256060578 - 0.7324274047478666im, -0.5474834394713616 - 1.4397439312132648e-18im, 0.5181314241778969 - 7.718609572936534e-17im, 0.02798959022341979 + 0.45123724019432776im, 0.027989590223419857 - 0.4512372401943277im, -0.2629974971645739 - 5.10965819315131e-19im, 0.1685440617420263 - 1.5332054970693233e-18im, -0.06343421095477318 + 2.677747793933849e-18im], ComplexF64[0.5701571203430421 - 0.4119985614377806im, 0.570157120343041 + 0.41199856143778046im, -0.38475678882052866 - 0.44756577888782123im, -0.3847567888205274 + 0.4475657788878202im, 0.423164609737971 - 0.40922553858979194im, 0.42316460973796943 + 0.40922553858979266im, -0.3720154037806365 + 0.3605505269546933im, -0.3720154037806359 - 0.360550526954693im, 0.17154883946058808 - 0.22205105315033843im, 0.17154883946058783 + 0.22205105315033838im, -0.10811868877667603 - 0.20850137104660763im, -0.10811868877667606 + 0.

In [33]:

S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
# display(S_set)

s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
# display(s_set)

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

...

"GLR_svd, Dtrun=5625, SU2 reduced to 71, minimal eigenvalue: 3.302368184276083e-7"

"Dtrun exceeds matrix size"

Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol


┌ Warning: Invariant subspace of dimension 168 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 182`); setting `howmany = 168`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 284 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 308`); setting `howmany = 284`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 315 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 349`); setting `howmany = 315`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 268 (up to requested tolerance `tol = 1.0e-12`), which is smaller than t

(Any[[1.3833658459006364, 0.5497151601683373, 0.45496033896761023, 0.14164388952956009, 0.12796101932112994, 0.09234049372796327, 0.06887090518759027, 0.01786392158747394, 0.0049386307992245835, 0.00020133397122969646], [0.509019726442333, 0.509019726442333, 0.2506765312584173, 0.250676531258417, 0.1870096521318885, 0.1870096521318885, 0.14947138379965852, 0.14947138379965846, 0.03975093647599152, 0.03975093647599144, 0.01563522450636746, 0.01563522450636746, 0.004650285689353908, 0.004650285689353905, 0.0028648221270427005, 0.0028648221270426966], [1.0950066089891008, 0.5055612456867157, 0.3614945669967845, 0.19950196242918744, 0.113902953690167, 0.09282896424716262, 0.08252925844666101, 0.06786835147053577, 0.06445646222817639, 0.058201746560533965, 0.025489221354212242, 0.016578801157715695, 0.006495182744622233, 0.002631589751132905, 0.001395555832275314, 0.0013655034562752766, 0.00036147828082242286], [0.8088930780281302, 0.8088930780281293, 0.23825533293897816, 0.2382553329389781

In [34]:
check_truncated_decomp_error=false;

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,true,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_eig)

eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_eig)


S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,true,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_svd)

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_svd)



if check_truncated_decomp_error

    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for ccc=1:pow-1
                euR=euR*euR0;
            end
            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(euR_set)
        euRs=euR_set_grouped[cc];
        evRs=evR_set_grouped[cc];
        evLs=evL_set_grouped[cc];
        spin=SPIN_eig_set_grouped[cc]
        if euRs==[]
        else
            for cs=1:length(euRs)
                euR0=euRs[cs];
                euR=euR0;
                for ccc=1:pow-1
                    euR=euR*euR0;
                end
                @tensor evR[:]:=U_fuse_chichi[-2,1,2]*evRs[cs][-1,1,2,-3,-4];
                evR=permute(evR,(1,2,3,),(4,));
                @tensor evL[:]:=evLs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                evL=permute(evL,(1,),(2,3,4,));
                M_temp1=M_temp1+evR*euR*evL*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(eur_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for ccc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(eur_set)
        eurs=eur_set_grouped[cc];
        evrs=evr_set_grouped[cc];
        evls=evl_set_grouped[cc];
        spin=spin_eig_set_grouped[cc]
        if eurs==[]
        else
            for cs=1:length(eurs)
                eur0=eurs[cs];
                eur=eur0;
                for ccc=1:pow-1
                    eur=eur*eur0;
                end

                M_temp1=M_temp1+evrs[cs]*eur*evls[cs]*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))




    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(S_set)
        Ss=S_set_grouped[cc];
        Us=U_set_grouped[cc];
        Vhs=Vh_set_grouped[cc];
        spin=SPIN_svd_set_grouped[cc]
        if Ss==[]
        else
            for cs=1:length(Ss)
                S=Ss[cs];
                @tensor U[:]:=U_fuse_chichi[-2,1,2]*Us[cs][-1,1,2,-3,-4];
                U=permute(U,(1,2,3,),(4,));
                @tensor Vh[:]:=Vhs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                Vh=permute(Vh,(1,),(2,3,4,));
                M_temp1=M_temp1+U*S*Vh*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(s_set)
        ss=s_set_grouped[cc];
        us=u_set_grouped[cc];
        vhs=vh_set_grouped[cc];
        spin=spin_svd_set_grouped[cc]
        if ss==[]
        else
            for cs=1:length(ss)
                s=ss[cs];
                M_temp1=M_temp1+us[cs]*s*vhs[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))
end

"group information:"

9-element Vector{Any}:
 UnitRange{Int64}[1:177]
 UnitRange{Int64}[1:300]
 UnitRange{Int64}[1:336]
 UnitRange{Int64}[1:294]
 UnitRange{Int64}[1:209]
 UnitRange{Int64}[1:115]
 UnitRange{Int64}[1:50]
 UnitRange{Int64}[1:11]
 ComplexF64[]

"group information:"

7-element Vector{Any}:
 UnitRange{Int64}[1:10]
 UnitRange{Int64}[1:16]
 UnitRange{Int64}[1:17]
 UnitRange{Int64}[1:14]
 UnitRange{Int64}[1:9]
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:1]

"group information:"

9-element Vector{Any}:
 UnitRange{Int64}[1:167]
 UnitRange{Int64}[1:281]
 UnitRange{Int64}[1:312]
 UnitRange{Int64}[1:265]
 UnitRange{Int64}[1:186]
 UnitRange{Int64}[1:99]
 UnitRange{Int64}[1:38]
 UnitRange{Int64}[1:10]
 Float64[]

"group information:"

7-element Vector{Any}:
 UnitRange{Int64}[1:10]
 UnitRange{Int64}[1:16]
 UnitRange{Int64}[1:17]
 UnitRange{Int64}[1:14]
 UnitRange{Int64}[1:9]
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:1]

In [35]:
        
function excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,VR,EU,VL,SPIN,N,k,DTrun_list)
    @assert pow==round((N-2)/2)
    x=input_transform*x;# do this calculation on cpu because the matrix 'input_transform' maybe large
    x=permute(x,(1,2,3,4,),());
    H_eff_x=x*0;
    cm=1;

    for cn=1:N
            coe=exp(-im*k*(cm-cn))
            H_eff_x0=H_eff_x*0;
            for c_sector=1:length(DTrun_list)
                for  c_comp=1:length(DTrun_list[c_sector])
                    #display(DTrun_list[c_sector][c_comp])
                    
                    H_eff_x0=H_eff_x0+coe*contract_H_eff_trun(x,A1,A2,Ag,VR[c_sector][c_comp],EU[c_sector][c_comp],VL[c_sector][c_comp],SPIN[c_sector],cm,cn,pow,N,DTrun_list[c_sector][c_comp]);
                end
            end
            H_eff_x=H_eff_x+H_eff_x0
    end
    x=permute(H_eff_x,(1,2,3,),(4,));
    x_output=output_transform*x;
    return x_output
end

function excitation_TrunTransOp_iterative_norm_eff(Ag,pow,N,k)
    @assert pow==round((N-2)/2)
    norm_eff=permute(TensorMap(randn,codomain(Ag)←codomain(Ag))*0,(1,2,3,4,5,6,),());
    cm=1;
    for cn=1:N
            coe=exp(-im*k*(cm-cn))
            norm_eff=norm_eff+coe*contract_norm_eff(Ag,cm,cn,N)
    end
    return norm_eff
end


excitation_TrunTransOp_iterative_norm_eff (generic function with 1 method)

In [36]:
function v_Transop(VL_comp,Ag,A1,A2,L)
    VL_comp=copy(VL_comp)
    for cl=1:L
        @tensor VL_comp[:]:=VL_comp[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*Ag[7,-5,6];
    end
    return VL_comp
end



v_Transop (generic function with 1 method)

In [37]:
function contract_H_eff_trun(x,A1,A2,Ag,VR_comp,EU_comp,VL_comp,spin_comp,cm,cn,pow,N,D_set)
    H_eff_x=x*0;
    L_Ea=cn-cm-1
    L_Eb=N-cn
    if L_Ea>=L_Eb
        L=L_Ea
        decomp_posit="Ea"
    elseif L_Ea<L_Eb
        L=L_Eb
        decomp_posit="Eb"
    end
    
    VL_comp=v_Transop(VL_comp,Ag,A1,A2,L-pow);
    VR_comp=VR_comp*EU_comp;
    

    
    if cm==cn # cm=cn=0
        @tensor ttt[:]:=VL_comp[9,-1,5,3,1]*A2[5,4,6,-3]*A1[3,2,7,4]*x[1,8,2,-4]*VR_comp[-2,6,7,8,9];
        H_eff_x=H_eff_x+ttt;
    elseif cn==2
        @tensor T1[:]:=VL_comp[-1,-2,5,3,1]*A2[5,4,-3,-6]*A1[3,2,-4,4]*Ag[1,-5,2];
        @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,-5,5,4]*VR_comp[1,3,5,-4,-6];
        @tensor T1T2x[:]:=T1[6,-1,3,4,5,-3]*T2[-2,3,4,2,1,6]*x[5,2,1,-4];
        H_eff_x=H_eff_x+T1T2x;
    elseif cn==N
        @tensor T1[:]:=VL_comp[-1,1,3,5,-2]*Ag'[1,-3,2]*A2[3,4,-4,2]*A1[5,-6,-5,4];
        @tensor T2[:]:=A2[-1,4,5,-5]*A1[-2,2,3,4]*Ag[-3,1,2]*VR_comp[-4,5,3,1,-6];
        @tensor T1xT2[:]:=T1[6,1,-1,3,4,2]*x[1,5,2,-4]*T2[3,4,5,-2,-3,6];
        H_eff_x=H_eff_x+T1xT2;
    else
        if  decomp_posit=="Ea"
            @tensor T1[:]:=VL_comp[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*x[7,-5,6,-6];
            @tensor T2[:]:=A2[-1,4,5,-5]*A1[-2,2,3,4]*Ag[-3,1,2]*VR_comp[-4,5,3,1,-6];
            @assert L_Eb>=1
            for cr=1:L_Eb
                @tensor T1[:]:=T1[-1,1,3,5,7,-6]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*Ag[7,-5,6];
            end
            @tensor T1T2[:]:=T1[4,-1,1,2,3,-4]*T2[1,2,3,-2,-3,4];
            H_eff_x=H_eff_x+T1T2;
        elseif decomp_posit=="Eb"
            @tensor T1[:]:=VL_comp[-1,-2,5,3,1]*A2[5,4,-3,-6]*A1[3,2,-4,4]*Ag[1,-5,2];
            @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,6,5,4]*x[-4,7,6,-6]*VR_comp[1,3,5,7,-5];
            @assert L_Ea>=1
            for cr=1:L_Ea
                @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,6,5,4]*Ag[-4,7,6]*T2[1,3,5,7,-5,-6];
            end
            @tensor T1T2[:]:=T1[4,-1,1,2,3,-3]*T2[-2,1,2,3,4,-4];
            H_eff_x=H_eff_x+T1T2;
        end
    end
    H_eff_x=H_eff_x*(2*spin_comp+1);
    return H_eff_x
end


    

contract_H_eff_trun (generic function with 1 method)

In [38]:
function contract_norm_eff(AL,cm,cn,N)

    mps=create_puMPS(AL,N);
    mps1=mps;
    mps2=mps;
    if cm==cn # cm=cn=1
        E=contiguous_TM(mps1,mps2,2:N)
        Id=id(space(Ag,3));
        @tensor norm_eff[:]:=E[-2,-5,-1,-4]*Id[-3,-6];
    elseif cn==2
        E=contiguous_TM(mps1,mps2,3:N);
        @tensor norm_eff[:]:=mps2[1][1,-4,-3]*mps1[2]'[-2,2,-6]*E[2,-5,-1,1];
    elseif cn==N
        E=contiguous_TM(mps1,mps2,2:N-1);
        @tensor norm_eff[:]:=mps2[1][-5,1,-3]*E[-2,1,2,-4]*mps1[N]'[2,-1,-6]
    else
        T1=mps2[1];
        T2=contiguous_TM(mps1,mps2,2:(cn-1))
        T3=mps1[cn]';
        T4=contiguous_TM(mps1,mps2,(cn+1):N)
        @tensor norm_eff[:]:=T1[6,1,-3]*T2[-2,1,3,-4]*T3[3,4,-6]*T4[4,-5,-1,6];
    end
    return norm_eff
end



contract_norm_eff (generic function with 1 method)

In [39]:
function contiguous_TM(mps1,mps2,positions)
    if length(positions)==1
        @tensor E[:]:=mps1[positions[1]]'[-1,-3,3]*mps2[positions[1]][-2,-4,3];
    elseif length(positions)>1
        @tensor E[:]:=mps1[positions[1]]'[-1,-3,3]*mps2[positions[1]][-2,-4,3];
        for ci=positions[2]:positions[end]
            @tensor E[:]:=E[-1,-2,1,2]*mps1[ci]'[1,-3,3]*mps2[ci][2,-4,3];
        end
    end
    return E
end

contiguous_TM (generic function with 1 method)

In [40]:

function create_puMPS(A,N)
    mps=Vector{Any}(undef, N);
    for c1=1:N
        mps[c1]=A
    end
    return mps
end

create_puMPS (generic function with 1 method)

In [41]:
function solve_ITEBD_excitation_TrunTransOp_iterative(Ag,A1,A2,n_E,N,kset,ES_sectors,pow,U,S,Vt,SPIN_group,DTrun_group)
    
    Eset=Matrix{Any}(undef, length(kset),length(ES_sectors)); 

    for kk=1:length(kset)

        ck=kset[kk]
        k=2*pi*ck/N
        

        norm_eff=excitation_TrunTransOp_iterative_norm_eff(Ag,pow,N,k) # put it on cpu because this matrix maybe large
        norm_eff=permute(norm_eff,(1,2,3,),(4,5,6,))
        uu,ss,vvt=tsvd(norm_eff, trunc=truncerr(0.0000001));
        norm_eff=[]#clear this big matrix
        input_transform=vvt';
        output_transform=uu';
        output_transform=output_transform;
        output_transform=pinv(ss)*output_transform;

        for sector_ind=1:length(ES_sectors)
            SPIN=ES_sectors[sector_ind];
            sectr=Irrep[SU₂](SPIN);
            display("sector "*"k="*string(ck)*", spin="*string(SPIN)*":");

            v_init=TensorMap(randn,domain(input_transform), SU₂Space(SPIN=>1));
            
            excitation_iterative(x)=excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,U,S,Vt,SPIN_group,N,k,DTrun_group)
            
            @time Es,_=eigsolve(excitation_iterative, v_init, n_E,:LM,Arnoldi());
            Eset[kk,sector_ind]=Es
            display(Es)
        end

    end


    return kset, Eset
end


solve_ITEBD_excitation_TrunTransOp_iterative (generic function with 1 method)

In [42]:
for cc=1:length(S_set_grouped)
    for cc2=1:length(S_set_grouped[cc])
        display(diag(convert(Array,S_set_grouped[cc][cc2])))
    end
end

167-element Vector{ComplexF64}:
   9.942462557414665e-5 + 0.0im
   4.869348316430439e-5 + 0.0im
    3.53719063939058e-5 + 0.0im
  2.2173492086850725e-5 + 0.0im
   1.345829404895034e-5 + 0.0im
   9.504264945863726e-6 + 0.0im
   8.938086949876201e-6 + 0.0im
    5.69950729754155e-6 + 0.0im
   4.299510146731962e-6 + 0.0im
  3.5080634124749112e-6 + 0.0im
                        ⋮
 1.0537854067395295e-11 + 0.0im
  8.293410005692105e-12 + 0.0im
   7.63858638461525e-12 + 0.0im
  6.324353087247019e-12 + 0.0im
  4.358729498094546e-12 + 0.0im
   3.64638373037801e-12 + 0.0im
 3.1945176564753353e-12 + 0.0im
 2.0756376435208603e-12 + 0.0im
 1.1225112252648805e-12 + 0.0im

562-element Vector{ComplexF64}:
   6.240709889110204e-5 + 0.0im
   6.240709889110204e-5 + 0.0im
  5.6475012891903234e-5 + 0.0im
  5.6475012891903234e-5 + 0.0im
  2.9068323168354636e-5 + 0.0im
  2.9068323168354636e-5 + 0.0im
   2.369701778807869e-5 + 0.0im
   2.369701778807869e-5 + 0.0im
  2.3197944370007374e-5 + 0.0im
  2.3197944370007374e-5 + 0.0im
                        ⋮
 2.6721004681054356e-12 + 0.0im
 2.2361818626193775e-12 + 0.0im
 2.2361818626193775e-12 + 0.0im
 1.8391539282427295e-12 + 0.0im
 1.8391539282427295e-12 + 0.0im
 1.5757920546130365e-12 + 0.0im
 1.5757920546130365e-12 + 0.0im
 1.2968629109301916e-12 + 0.0im
 1.2968629109301916e-12 + 0.0im

936-element Vector{ComplexF64}:
    9.08623419486046e-5 + 0.0im
    9.08623419486046e-5 + 0.0im
    9.08623419486046e-5 + 0.0im
  4.4160468772438006e-5 + 0.0im
  4.4160468772438006e-5 + 0.0im
  4.4160468772438006e-5 + 0.0im
   3.759934534732787e-5 + 0.0im
   3.759934534732787e-5 + 0.0im
   3.759934534732787e-5 + 0.0im
   2.422850946974379e-5 + 0.0im
                        ⋮
 1.4880745134064933e-12 + 0.0im
 1.4880745134064933e-12 + 0.0im
 1.4880745134064933e-12 + 0.0im
 1.3169322936901674e-12 + 0.0im
 1.3169322936901674e-12 + 0.0im
 1.3169322936901674e-12 + 0.0im
 1.0188206029077713e-12 + 0.0im
 1.0188206029077713e-12 + 0.0im
 1.0188206029077713e-12 + 0.0im

1060-element Vector{ComplexF64}:
   7.163736272579111e-5 + 0.0im
   7.163736272579111e-5 + 0.0im
   7.163736272579111e-5 + 0.0im
   7.163736272579111e-5 + 0.0im
   6.540060580070206e-5 + 0.0im
   6.540060580070206e-5 + 0.0im
   6.540060580070206e-5 + 0.0im
   6.540060580070206e-5 + 0.0im
  2.9671911602028748e-5 + 0.0im
  2.9671911602028748e-5 + 0.0im
                        ⋮
 1.6602348520514633e-12 + 0.0im
  1.225882029010534e-12 + 0.0im
  1.225882029010534e-12 + 0.0im
  1.225882029010534e-12 + 0.0im
  1.225882029010534e-12 + 0.0im
  1.001313079831602e-12 + 0.0im
  1.001313079831602e-12 + 0.0im
  1.001313079831602e-12 + 0.0im
  1.001313079831602e-12 + 0.0im

930-element Vector{ComplexF64}:
   4.220017519137451e-5 + 0.0im
   4.220017519137451e-5 + 0.0im
   4.220017519137451e-5 + 0.0im
   4.220017519137451e-5 + 0.0im
   4.220017519137451e-5 + 0.0im
   2.077225904588184e-5 + 0.0im
   2.077225904588184e-5 + 0.0im
   2.077225904588184e-5 + 0.0im
   2.077225904588184e-5 + 0.0im
   2.077225904588184e-5 + 0.0im
                        ⋮
 1.3722225866327816e-12 + 0.0im
 1.3722225866327816e-12 + 0.0im
 1.3722225866327816e-12 + 0.0im
 1.3722225866327816e-12 + 0.0im
 1.0636690833941157e-12 + 0.0im
 1.0636690833941157e-12 + 0.0im
 1.0636690833941157e-12 + 0.0im
 1.0636690833941157e-12 + 0.0im
 1.0636690833941157e-12 + 0.0im

594-element Vector{ComplexF64}:
   9.979992427290504e-6 + 0.0im
   9.979992427290504e-6 + 0.0im
   9.979992427290504e-6 + 0.0im
   9.979992427290504e-6 + 0.0im
   9.979992427290504e-6 + 0.0im
   9.979992427290504e-6 + 0.0im
  5.8965139855728564e-6 + 0.0im
  5.8965139855728564e-6 + 0.0im
  5.8965139855728564e-6 + 0.0im
  5.8965139855728564e-6 + 0.0im
                        ⋮
 1.3100584077651031e-12 + 0.0im
 1.3100584077651031e-12 + 0.0im
 1.3100584077651031e-12 + 0.0im
 1.0676907278291143e-12 + 0.0im
 1.0676907278291143e-12 + 0.0im
 1.0676907278291143e-12 + 0.0im
 1.0676907278291143e-12 + 0.0im
 1.0676907278291143e-12 + 0.0im
 1.0676907278291143e-12 + 0.0im

266-element Vector{ComplexF64}:
   8.693025367993127e-8 + 0.0im
   8.693025367993127e-8 + 0.0im
   8.693025367993127e-8 + 0.0im
   8.693025367993127e-8 + 0.0im
   8.693025367993127e-8 + 0.0im
   8.693025367993127e-8 + 0.0im
   8.693025367993127e-8 + 0.0im
  5.6234841700383346e-8 + 0.0im
  5.6234841700383346e-8 + 0.0im
  5.6234841700383346e-8 + 0.0im
                        ⋮
 2.4239505190919246e-12 + 0.0im
 2.4239505190919246e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im
 1.4006884834178444e-12 + 0.0im

80-element Vector{ComplexF64}:
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
 3.8942669852784356e-10 + 0.0im
  3.120526159562171e-10 + 0.0im
  3.120526159562171e-10 + 0.0im
                        ⋮
 3.2906920045437895e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im
 2.2906763455268863e-12 + 0.0im

In [43]:
Dtrun_method="svds";
ES_sectors=[0,1/2,1,3/2,2,5/2];
kset=0:N-1;
#kset=0:0
Eset=[];
Trun_err=0;
DTrun=0;
print("calculate ES for N="*string(N));
display("kset="*string(kset));
pow=round((N-2)/2);



if Dtrun_method=="eigs"
    DTrun=length(group_numbers(SPIN_eig_set));
    display("DTrun="*string(DTrun));

    euRs=abs.(group_numbers(euR_set));
    Trun_err=(minimum(euRs)/maximum(euRs))^pow;


    euR_pow=deepcopy(euR_set_grouped);
    for ca=1:length(euR_pow)
        for cb=1:length(euR_pow[ca])
            euR_pow[ca][cb]=euR_pow[ca][cb]^Int(pow);
        end
    end

    H_renorm=maximum(euRs);

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1/H_renorm,O2,EH_n,N,kset,ES_sectors,pow,evR_set_grouped,euR_pow,evL_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig)

elseif Dtrun_method=="svds"
    DTrun=length(group_numbers(SPIN_svd_set));
    display("DTrun="*string(DTrun));

    Ss=abs.(group_numbers(S_set));
    Trun_err=(minimum(Ss)/maximum(Ss));

    H_renorm=maximum(Ss)^(1/pow);

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1/H_renorm,O2,EH_n,N,kset,ES_sectors,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd)
end

ES_filenm="ES_"*Dtrun_method*"_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*"_N"*string(N)*".mat";
matwrite(ES_filenm, Dict(
    "kset" => convert(Vector,kset),
    "ES_sectors" => ES_sectors,
    "Eset" => Eset,
    "Trun_err"=>Trun_err,
    "DTrun"=>DTrun
); compress = false)


"kset=0:7"

"DTrun=1358"

"sector k=0, spin=0.0:"

17-element Vector{ComplexF64}:
 3.2750621229201876e-6 - 1.2391880140259053e-6im
  3.274903388069776e-6 + 1.2391078282888295e-6im
  9.144799316077222e-7 + 2.599361170214005e-7im
  9.143468841911267e-7 - 2.600461890375939e-7im
  3.616842301219312e-7 - 3.484613135583785e-11im
  3.328836756895109e-7 + 4.958402128297144e-13im
  3.100956045934723e-7 + 9.38138673028305e-12im
 2.4883419648130854e-7 - 7.152414895878873e-8im
  2.488036060475399e-7 + 7.152310914665161e-8im
 1.8084062075600456e-7 - 5.190726062391112e-12im
  9.502702194002437e-8 + 2.8339566880475894e-9im
     9.490829766994e-8 - 2.8190166395722876e-9im
  6.904306140863916e-8 + 6.326737504328282e-12im
 5.3017211119077025e-8 + 2.7974275662454202e-8im
  5.301178839826699e-8 - 2.7972075097653464e-8im
 5.1579806746175065e-9 + 7.030754110967916e-9im
  5.160332228666722e-9 - 7.018724107881266e-9im

"sector k=0, spin=0.5:"

...

"sector k=7, spin=2.0:"

18-element Vector{ComplexF64}:
  9.089396565526002e-7 + 2.4248126236452044e-11im
  3.049002432980744e-7 - 5.40813336182489e-12im
 2.1376334306790743e-7 + 3.975920944345375e-11im
 1.6023023926957466e-7 - 6.918863843254781e-11im
 1.2644401166738605e-7 + 5.3653548722813945e-11im
  9.334173744040307e-8 + 1.7646212163442643e-8im
  9.329036052146382e-8 - 1.7661463710634908e-8im
  8.265048291639116e-8 + 1.419586107540538e-11im
  5.924862458567792e-8 + 1.3290748385875362e-8im
  5.922079476455939e-8 - 1.329053509712449e-8im
 4.8440829561408585e-8 - 5.506005427852754e-11im
 4.1100148522234536e-8 + 1.1882694962238166e-10im
  3.671054037282375e-8 - 2.946029434222262e-11im
 2.9426056314819207e-8 - 8.447665253570759e-9im
  2.940251823954312e-8 + 8.420254448572158e-9im
 1.8454366437003013e-8 + 1.7031917052992356e-11im
 1.1773358050565057e-8 + 4.4872829130830715e-13im
 1.1068649510869986e-8 - 8.274004050385047e-13im

"sector k=7, spin=2.5:"

10-element Vector{ComplexF64}:
  3.983417334024814e-7 + 8.558575033332411e-14im
  9.267008314706982e-8 + 1.0671093021506191e-11im
  8.207835271798956e-8 - 3.057249634874091e-11im
  6.025591822213966e-8 + 5.071641287655399e-13im
 3.3942101985038136e-8 + 4.5235076990195977e-10im
 2.8888605973750722e-8 - 3.111776067889142e-10im
 2.0465404112965685e-8 + 4.3135592533838735e-12im
 1.5186822654620736e-8 - 1.6036626295009973e-11im
   9.78082581182379e-9 + 3.2728589974191093e-12im
  7.071145028480439e-9 + 4.1471628188166286e-11im

calculate ES for N=8179.168428 seconds (82.10 M allocations: 16.579 GiB, 11.95% gc time, 1.02% compilation time)
384.902479 seconds (171.69 M allocations: 31.382 GiB, 11.06% gc time, 0.20% compilation time)
538.784958 seconds (249.03 M allocations: 42.802 GiB, 13.46% gc time, 0.37% compilation time)
500.692608 seconds (251.27 M allocations: 41.605 GiB, 13.63% gc time, 0.39% compilation time)
352.903000 seconds (189.70 M allocations: 30.690 GiB, 17.03% gc time, 0.22% compilation time)
210.398398 seconds (113.98 M allocations: 18.142 GiB, 24.77% gc time, 0.01% compilation time)
170.001506 seconds (73.35 M allocations: 14.467 GiB, 14.00% gc time, 0.06% compilation time)
341.893419 seconds (175.19 M allocations: 31.933 GiB, 12.56% gc time, 0.00% compilation time)
328.900433 seconds (249.05 M allocations: 42.836 GiB, 12.31% gc time, 0.10% compilation time)
313.467479 seconds (251.29 M allocations: 41.654 GiB, 12.28% gc time)
214.898815 seconds (189.90 M allocations: 30.711 GiB, 16.09% gc ti